In [17]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import operator
from tqdm import tqdm
import os

In [3]:
from text_processing import get_text_map


In [4]:
USER_NAME = "littel_musician"

In [13]:
texts_lenta = pd.read_csv("./colloc/music_lenta.csv")
texts_lenta.iloc[69]['text']

'Четыре ранее не издававшиеся песни британской поп-группы Spice Girls просочились в интернет, сообщает Variety. Композиции A Day in Your Life, Pain Proof, If It’s Lovin’ On Your Mind и Right Back at Ya появились на онлайн-платформе Soundcloud. Позже треки были удалены. Как поясняет издание, участницы Spice Girls записали эти песни, когда работали над альбомом Forever в 2000 году. Они не вошли в финальный трек-лист пластинки, и певицы не планировали выпускать их отдельно. По одной из версий, уточняет Variety, композиция Right Back at Ya все-таки попала в альбом, но в другой вариации. Пластинка Forever стала третьей и последней в карьере коллектива. Диск попал в первую десятку музыкальных рейтингов Великобритании, Германии, Австрии, Канады, Австралии и Италии. Группа Spice Girls образовалась в 1994 году и продолжала постоянно выступать до 2001-го. Несколько раз участницы коллектива\xa0— Виктория Бекхэм, Эмма Бантон, Мелани Чисхолм, Мелани Браун, Джери Холлиуэлл\xa0— снова объединялись, в

In [73]:
examined_text = get_text_map(texts_lenta.iloc[69]['texts_3000'], raw_text_input=True)

In [5]:
text_db = pd.read_csv(USER_NAME + "_text_db.csv",header = None)
text_db.head()

,0,1,2,3,4,5,6,7,8,9
0,0.6118,0.75,0.200000,0.200000,0.024267,0.030002,0.000000,0.071312,0.191511,1.0
1,0.8013,0.63,0.000000,0.071429,0.015801,0.032847,0.004245,0.145502,0.212638,1.0
2,0.5788,0.70,0.333333,0.266667,0.031384,0.032255,0.000000,0.130564,0.209426,1.0
3,0.4068,0.55,0.242424,0.303030,0.021535,0.010711,0.000000,0.054207,0.195904,1.0


In [6]:
sent_db = pd.read_csv(USER_NAME + "_sentence_db.csv",header = None)
sent_db.head()

,0,1,2,3,4,5,6,7
0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.209091,1.0
1,0.018519,0.000000,0.021353,0.000000,0.0,0.042707,0.347170,1.0
2,0.034483,0.034483,0.000000,0.063484,0.0,0.143760,0.217857,1.0
3,0.000000,0.000000,0.000000,0.000000,0.0,0.177674,0.219048,1.0
4,0.000000,0.000000,0.000000,0.000000,0.0,0.107480,0.205263,1.0


In [7]:
word_db = pd.read_csv(USER_NAME +  "_word_db.csv",header = None)
word_db.head()

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,300,301,302
0,72.0,155.10,0.019617,0.027856,-0.098140,0.018417,-0.060545,0.052610,-0.008416,0.019377,...,-0.045566,-0.019588,-0.051453,-0.000417,0.015935,0.018377,-0.049183,0.040319,0.087135,1.0
1,1.0,17.10,-0.059523,0.012793,0.034546,-0.019653,-0.059405,-0.108276,-0.020982,0.038993,...,0.080227,0.072730,0.008212,0.018211,-0.058539,-0.046240,-0.067024,-0.060923,-0.018587,1.0
2,73.0,483.30,-0.012936,-0.003571,-0.036513,-0.125645,-0.025398,-0.118884,-0.007071,-0.008625,...,0.069345,0.021790,-0.002086,0.007569,-0.078134,0.048388,-0.009063,0.001203,-0.019108,1.0
3,71.0,16049.95,0.052736,0.012474,-0.023544,0.048177,-0.050497,-0.054198,0.043881,-0.023119,...,0.015864,0.024000,0.093944,0.019959,-0.034256,-0.002028,-0.055652,-0.034802,-0.000229,1.0
4,81.0,1129.20,-0.024112,-0.021304,0.030345,-0.001343,-0.000653,-0.011318,-0.013344,0.044790,...,0.007491,-0.008148,0.051118,-0.022130,0.003563,0.006337,0.010453,0.024994,-0.046207,1.0


with open ("text_map_improved_example.json", "r", encoding = "utf-8") as f:
    examined_text = json.load(f)

In [8]:
words_dict = []
with open("colloc_lit.txt", "r", encoding = "utf-8") as f:
    for line in f.readlines():
        words_dict.append(line[:-1])

In [9]:
words_dict[0]

'оказываться'

In [46]:
my_list = [ ('John', 40), ('Dana', 35), ('Betty', 10), ('Robby', 8), ('John', 20) ]
my_list.sort(key = operator.itemgetter(1))
print(my_list)

[('Robby', 8), ('Betty', 10), ('John', 20), ('Dana', 35), ('John', 40)]


In [32]:
def get_top_similar_vectors_from_marked_db(analyzed_text_map,marked_text_db, marked_sent_db, marked_word_db, words_indexed_dict):
    
    similarity_json = {"similar_texts":[],"simiar_sentences":[], "similar_collocations":[]}
    #text_similarity
    text_vector = []
    text_vector.append(analyzed_text_map['lix'])
    text_vector.append(analyzed_text_map['ttr'])
    text_vector.extend(analyzed_text_map['sent_properties'])
    list_of_texts_similarities = []
    for text_ind in range(len(marked_text_db)):
        #print(text_vector)
        #target_var = list(marked_text_db.iloc[2][-1:]) WILL NEED WHEN PREDICT!
        v1 = np.array(text_vector).reshape(1, -1)
        v2 = np.array(list(marked_text_db.iloc[text_ind][:-1])).reshape(1, -1)
        sim = cosine_similarity(v1,v2)
        #print(sim[0][0])
        if sim[0][0] > 0.6:
            list_of_texts_similarities.append((sim[0][0], {"TEXT_IND_in_marked_db":text_ind, "VECTOR":list(marked_text_db.iloc[text_ind][:-1])}))
            
    list_of_texts_similarities.sort(key = operator.itemgetter(0), reverse= True)   
    similarity_json['similar_texts'] = list_of_texts_similarities[:5]
    
    list_of_all_sentences_similarity = []
    for sentence in analyzed_text_map['sentences_map']:
        current_sentence = []
        current_sentence.append(sentence['spec_sentence_features']['negation'])
        current_sentence.append(sentence['spec_sentence_features']['coreference'])
        current_sentence.append(sentence['spec_sentence_features']['vozvr_verb'])
        current_sentence.append(sentence['spec_sentence_features']['prich'])
        current_sentence.append(sentence['spec_sentence_features']['deepr'])
        current_sentence.append(sentence['spec_sentence_features']['case_complexity'])
        current_sentence.append(sentence['spec_sentence_features']['mean_depend_length'])
        v1 = np.array(current_sentence).reshape(1, -1)
        list_of_current_sent_similarities = []
        for sent_ind in range(len(marked_sent_db)):
            v2 = np.array(list(marked_sent_db.iloc[sent_ind][:-1])).reshape(1, -1)
            sim = cosine_similarity(v1,v2)
            if sim[0][0] > 0.6:
                list_of_current_sent_similarities.append((sim[0][0], {"sent_ind_in_marked_db":sent_ind, "sent_vector":list(marked_sent_db.iloc[sent_ind][:-1])}))
        list_of_current_sent_similarities.sort(key = operator.itemgetter(0), reverse= True)
        list_of_all_sentences_similarity.append(list_of_current_sent_similarities[:5])
    similarity_json['simiar_sentences'] = list_of_all_sentences_similarity
    
    list_of_all_words_similarity = []
    for sentence in analyzed_text_map['sentences_map']:
        for colloc in sentence['collocation_vectors_list']:
            current_word_similarity = []
            #print(colloc[1][0])
            freq_vec = np.array([colloc[1][1], colloc[1][2]]).reshape(1,-1)
            colloc_w2v = np.array(colloc[2]).reshape(1,-1)
            for db_word_index in range(len(marked_word_db)):
                compared_raw_word_vect = marked_word_db.iloc[db_word_index][:-1]
                
                comp_word2vec = compared_raw_word_vect[2:]
                #print(db_word_index, len(comp_word2vec))
                comp_colloc_w2v = np.array(comp_word2vec).reshape(1,-1)
                w2v_sim = cosine_similarity(colloc_w2v,comp_colloc_w2v)
                #print("w2v", sim)
                
               
                comp_freq_vec = np.array([compared_raw_word_vect[0], compared_raw_word_vect[1]]).reshape(1,-1)
                sim_freq = cosine_similarity(freq_vec,comp_freq_vec)
                #print("freq", sim_freq)
                
                w2v_sim = w2v_sim[0][0]
                sim_freq = sim_freq[0][0]
                
                average_similarity = w2v_sim*0.8 + sim_freq*0.2
                
                word_comp_json = {"colloc_db_index":db_word_index, "sim_colloc":words_indexed_dict[db_word_index],"w2v_sim":w2v_sim,"freqence_similarity":sim_freq}
                if average_similarity> 0.6:
                    current_word_similarity.append((word_comp_json,average_similarity))
            current_word_similarity.sort(key = operator.itemgetter(1), reverse= True)
            list_of_all_words_similarity.append((colloc[1][0],current_word_similarity[:5]))
    similarity_json['similar_collocations'] = list_of_all_words_similarity
    return similarity_json
    


In [33]:
text_sim_list = get_top_similar_vectors_from_marked_db(examined_text,text_db,sent_db,word_db,words_dict)

In [36]:
handel_texts_index = [0,
17,
26,
32]
for text_ind in tqdm(range(len(texts_lenta))):
    if text_ind not in handel_texts_index:
        examined_text = get_text_map(texts_lenta.iloc[text_ind]['text'], raw_text_input=True)
        text_sim_list = get_top_similar_vectors_from_marked_db(examined_text,text_db,sent_db,word_db,words_dict)
        with open(os.path.join('./music_similarity',str(text_ind) + ".json"), "w") as f:
            json.dump(text_sim_list, f, indent = 2, ensure_ascii=False)








  0%|          | 0/5099 [00:00<?, ?it/s]





  0%|          | 2/5099 [00:18<13:17:01,  9.38s/it]





  0%|          | 3/5099 [00:33<15:24:04, 10.88s/it]





  0%|          | 4/5099 [01:03<23:27:25, 16.57s/it]





  0%|          | 5/5099 [01:32<28:59:27, 20.49s/it]





  0%|          | 6/5099 [01:56<30:22:16, 21.47s/it]





  0%|          | 7/5099 [02:10<27:03:11, 19.13s/it]

KeyboardInterrupt: 

In [22]:
text_sim_list['similar_texts']

[(0.9778409710955541,
  {'TEXT_IND_in_marked_db': 0,
   'VECTOR': [0.6118,
    0.75,
    0.2,
    0.2,
    0.024266827607358692,
    0.030002295475404474,
    0.0,
    0.07131167644959037,
    0.191510992676087]}),
 (0.9534319917095977,
  {'TEXT_IND_in_marked_db': 2,
   'VECTOR': [0.5788000000000001,
    0.7,
    0.33333333333333326,
    0.2666666666666666,
    0.03138418932732309,
    0.032255172907681075,
    0.0,
    0.13056425505926705,
    0.20942576798157186]}),
 (0.9313044385335004,
  {'TEXT_IND_in_marked_db': 1,
   'VECTOR': [0.8013,
    0.63,
    0.0,
    0.07142857142857142,
    0.01580082003640032,
    0.03284652291377129,
    0.004244871454581902,
    0.14550185451989586,
    0.21263766295378408]}),
 (0.9189226898086713,
  {'TEXT_IND_in_marked_db': 3,
   'VECTOR': [0.4068,
    0.55,
    0.24242424242424246,
    0.30303030303030304,
    0.021535155564896602,
    0.01071091529831973,
    0.0,
    0.054207045977692284,
    0.19590366066250947]})]

In [24]:
text_sim_list['simiar_sentences']

[[(0.8991200500184969,
   {'sent_ind_in_marked_db': 14,
    'sent_vector': [0.05555555555555556,
     0.0,
     0.06912763848250579,
     0.0,
     0.0,
     0.0,
     0.2823529411764706]}),
  (0.8128539719980905,
   {'sent_ind_in_marked_db': 13,
    'sent_vector': [0.047619047619047616, 0.0, 0.0, 0.0, 0.0, 0.0, 0.205]}),
  (0.7747220194681459,
   {'sent_ind_in_marked_db': 1,
    'sent_vector': [0.01851851851851852,
     0.0,
     0.02135345373297045,
     0.0,
     0.0,
     0.0427069074659409,
     0.34716981132075475]}),
  (0.7706509750399892,
   {'sent_ind_in_marked_db': 8,
    'sent_vector': [0.0,
     0.0,
     0.04857791214148391,
     0.037409512160146506,
     0.0,
     0.07481902432029298,
     0.2708333333333334]}),
  (0.7635884803791589,
   {'sent_ind_in_marked_db': 15,
    'sent_vector': [0.0,
     0.05555555555555556,
     0.13381846042771664,
     0.0,
     0.0,
     0.11464204212262835,
     0.17058823529411762]})],
 [(0.9998116175147458,
   {'sent_ind_in_marked_db': 4,

In [34]:
for sim in text_sim_list['similar_collocations']:
    print (sim)
    print()

('евгений', [({'colloc_db_index': 23, 'sim_colloc': 'борис', 'w2v_sim': 0.7604470643013435, 'freqence_similarity': 0.9966534773353185}, 0.8076883469081385), ({'colloc_db_index': 33, 'sim_colloc': 'георгий', 'w2v_sim': 0.731150744712298, 'freqence_similarity': 0.9900133852590709}, 0.7829232728216526), ({'colloc_db_index': 84, 'sim_colloc': 'виктория', 'w2v_sim': 0.5819785619906627, 'freqence_similarity': 0.9877247503148777}, 0.6631277996555057), ({'colloc_db_index': 86, 'sim_colloc': 'альберт', 'w2v_sim': 0.5398656367875393, 'freqence_similarity': 0.9935146096866537}, 0.6305954313673622)])

('не', [({'colloc_db_index': 132, 'sim_colloc': 'не', 'w2v_sim': 1.0000000000000004, 'freqence_similarity': 0.9999999999999999}, 1.0000000000000004), ({'colloc_db_index': 123, 'sim_colloc': 'не мочь', 'w2v_sim': 0.634628076223879, 'freqence_similarity': 0.9963110175691335}, 0.7069646644929299), ({'colloc_db_index': 108, 'sim_colloc': 'не удаваться', 'w2v_sim': 0.5416087774661444, 'freqence_similarity

# В ПРОЦЕСЕ РЕКОММЕНДАЦИИ ПОЛЬЗОВАТЬСЯ ИНДЕКСАМИ РАЗМЕЧЕННЫХ ПРЕДЛОЖЕНИЙ ИЗ БАЗЫ ДАННЫХ
ТО ЕСТЬ В КАРТАХ ИМИЛАРИТИ ДОЛЖНЫ БЫТЬ СРАЗУ ИНДЕКСЫ ВСЕЙ ХЕРНИ

In [45]:
for text in text_sim_list:
    if text[1] == text_vector:
        print("yes")

yes
